In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import pickle as pkl

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
with open('datasets/normed_singletons_af_w_pza_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)
# with open('datasets/singletons_af_no_mpfs_graph_dict.pkl', 'rb') as f:
#     graph_dict = pkl.load(f)
# with open('datasets/singletons_no_af.pkl', 'rb') as f:
#     graph_dict = pkl.load(f)
    
print(len(graph_dict['train']) + len(graph_dict['test']))

664


### Parameters

Best param value ranges from WandB sweeps

- Cutoff distance = 11 - 12.5
- Dropout = 0.4-0.5
- Edge weights = "exp"
- Edge weight lambda = 2
- Hidden channels = 320
- Learning rate = 3.5e-5 - 4.5e-5
- Weight decay = 1e-6 - 1e-5

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)


# logging params (only used for wandb metrics)
n_samples = len(graph_dict['train']) + len(graph_dict['test'])
cutoff_distance = 12

# gcn params - from best wandb sweep
num_node_features = 16
batch_size = 256
hidden_channels = 320
dropout = 0.5

edge_weight_func = "exp"
edge_weight_lambda = 2

learning_rate = 4e-5
wd = 5e-5
epochs = 2000


wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [4]:
project = 'train-final-model'

In [ ]:
# run_name = f'Run 7 - {epochs} epochs - lr {learning_rate}'
run_name = f'No node chem feats - Run 1 - {epochs} epochs'

model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            cutoff_distance = cutoff_distance,
            edge_weight_func = edge_weight_func,
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = hidden_channels,
            learning_rate = learning_rate,
            wd = wd,
            dropout = dropout,
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            lambda_param= edge_weight_lambda,
            early_stop=False,
            # no_node_chem_feats=True,
            save_path= f'saved_models/{project}/{run_name}',
            wandb_params={
              'use_wandb': True, 
              'wandb_project': f'{project}', 
              'wandb_name': f'{run_name}',
              'n_samples': n_samples,
              'sweep': False
              }
        )

In [ ]:
# torch.save(model, f'saved_models/{project}/{run_name} - 0.80198 Test F1')

In [3]:
graph_dict['train']['pnca_mut_0']['graph'].dataset[0].x.size(1)

16